In [30]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import glob
import os

In [31]:
allFiles = sorted(glob.glob("*.txt"),key=os.path.getmtime) # includes all files into a single stucture,
# Sorted by modification time so that the first measurement is first...
print(allFiles)

['SHGData_ThzBlocked_07042017.txt', 'SHGData_Use0PostIt_07042017.txt', 'SHGData_Use3PostIt_07042017.txt', 'SHGData_Use2PostIt_07042017.txt', 'SHGData_Use1PostIt_07042017.txt', 'SHGData_Use4PostIt_07042017.txt']


In [8]:
frame = pd.DataFrame()
df = pd.DataFrame()

In [34]:
for i,file_ in enumerate(allFiles):
#    f = open(file_,'r')
#     for j,line in enumerate(f):
#         if j == 1:
#             date = line[14:22]
#             break
    df = pd.read_csv(file_,index_col = None, header=None, skiprows = 10, skipfooter = 1, delimiter = '\t')
#     frame[file_] = df[5]
    df['file']=file_
    df['timeDelay']= df[1]
    df['PDvalue'] = df[3]
#     df['date']=date
    if i < 1:
        frame = df
    else:
        frame = frame.append(df)

/Users/sanjaykhatri/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skip_footer; you can avoid this warning by specifying engine='python'.


In [33]:
c = 3*10**11; #Speed of Light in mm/s, Going to use this to convert spatial information to time. 
T1 = (499*0.000125 + 0.0229)*2*25.4/(10**-12*c)
def ConvertToTimeDelay(x,itr,offset):
    c = 3*10**11
    timeDelay = []
    for i in range(0,len(x)):
        timedelay = ((x[i]*itr + offset)*25.4*2/(10**-12*c))-T1
        timeDelay.append(timedelay)
    return(timeDelay)

In [42]:
times = pd.Series()
PDvalues = pd.Series()
AvgPD50 = pd.Series()
itr = 0.000125
offset = 0.023
for i in range(0,300):
    time = frame[1][i]
    PDvalue = frame[3][i]
    times = times.append(time)
    PDvalues = PDvalues.append(PDvalue)
#print(len(PDvalues))
    
times = times.values.reshape(300,len(allFiles))
PDvalues = PDvalues.values.reshape(300,len(allFiles)) 
timeDelay = ConvertToTimeDelay(times,itr,offset) # function converts to timedelay

MeanPD = PDvalues.mean(axis = 0) #Getting mean of all the data
StDevPD = PDvalues.std(axis = 0) # Getting Standard Deviation of the data
ErrOfMean = (StDevPD)/(300**0.5) # Standard error of mean stdev/sqrt(#ofdatapoints)
Background = MeanPD[0]
ActualPD = np.absolute(MeanPD - Background)
RealPD = np.delete(ActualPD,0)
LogPD = np.log10(np.delete(RealPD,-2))
THzEnergy = np.array([50,20,30,10])
LogTHz = np.log10(THzEnergy)
error = np.delete(StDevPD,-1)
Error = np.delete(error,-1)
print(LogTHz)
print(RealPD)

[ 1.69897     1.30103     1.47712125  1.        ]
[ 0.06637667  0.043325    0.04633167  0.04107567  0.03604967]


In [49]:
fit = np.polyfit(LogTHz,LogPD,1,w=Error)
f = np.poly1d(fit)
pPower = np.linspace(LogTHz[2],LogTHz[-1],100)
f_new = f(pPower)
print(f_new)
print(f)

[-1.28458322 -1.28635171 -1.2881202  -1.28988869 -1.29165719 -1.29342568
 -1.29519417 -1.29696266 -1.29873116 -1.30049965 -1.30226814 -1.30403663
 -1.30580513 -1.30757362 -1.30934211 -1.3111106  -1.3128791  -1.31464759
 -1.31641608 -1.31818457 -1.31995307 -1.32172156 -1.32349005 -1.32525854
 -1.32702704 -1.32879553 -1.33056402 -1.33233251 -1.33410101 -1.3358695
 -1.33763799 -1.33940648 -1.34117497 -1.34294347 -1.34471196 -1.34648045
 -1.34824894 -1.35001744 -1.35178593 -1.35355442 -1.35532291 -1.35709141
 -1.3588599  -1.36062839 -1.36239688 -1.36416538 -1.36593387 -1.36770236
 -1.36947085 -1.37123935 -1.37300784 -1.37477633 -1.37654482 -1.37831332
 -1.38008181 -1.3818503  -1.38361879 -1.38538729 -1.38715578 -1.38892427
 -1.39069276 -1.39246126 -1.39422975 -1.39599824 -1.39776673 -1.39953523
 -1.40130372 -1.40307221 -1.4048407  -1.4066092  -1.40837769 -1.41014618
 -1.41191467 -1.41368317 -1.41545166 -1.41722015 -1.41898864 -1.42075714
 -1.42252563 -1.42429412 -1.42606261 -1.42783111 -1.

In [45]:
print(LogTHz)

[ 1.69897     1.30103     1.47712125  1.        ]


In [46]:
print(LogPD)

[-1.17798456 -1.36326143 -1.33412208 -1.44309875]


In [48]:
print(Error)

[ 0.0056269   0.0050363   0.00436396  0.00550762]
